In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.utils import to_categorical

def q_network():
    
    '''
    Creamos nuestro agente, en este caso es tan sencillo como una NN que toma como entrada el estado
    y devuelve la acción a realizar. Está compuesta por una capa de entrada, una capa oculta fully connected de 128 nodos
    y una capa de salida de 2 nodos (correspondiente a las 2 posibles acciones del entorno cartpole)
    '''
    # Inicializamos la red neuronal
    net = Sequential(name='Fully-Connected-Network')

    # Añadimos la capa oculta y le indicamos que el input 
    # debe corresponder al espacio de observación
    # Inicializamos los pesos de forma aleatoria uniformemente distribuida
    # Indicamos una función de activación para esa capa oculta de tipo ReLU

    net.add(Dense(
        128, 
        input_shape=(ENV.observation_space.n,), 
        name='Hidden',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        activation='relu'
            )
        )

    # Añadimos una última capa de salida con 2 nodos y función de activación relu.
    # Esta función de activación nos devuelve la Q de cada acción en el estado introducido en el input
    
    net.add(Dense(
        ENV.action_space.n, 
        name='Output',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        activation='relu')
       )

    # Obtenemos información de esta red 
    net.summary()

    # La compilamos para su posterior entrenamiento con el optimizador Adam y
    # la función de pérdida de tipo MeanSquaredError() típica de ejercicios de regresión (como este)
    # Y utilizada por DeepMind en su paper sobre DQN
    
    net.compile(
        optimizer=Adam(), 
        loss=MeanSquaredError(), 
        metrics=['acc'])
    
    return net

In [7]:
from numpy import argmax
from random import random

def get_action_and_value_epsilon_greedy(q_values):
    
    '''
    Ejecuta una política epsilon-greedy para seleccionar la acción a realizar
    @ q_values: list --> Lista con los valores de Q predichos por la red neuronal 
    '''
    
    # Actuar random
    if random() < EPSILON:
        
        action, action_q_value = get_random_action_and_value(q_values)
            
    # Actuar Greedy:
    else:
        
        action, action_q_value = get_best_action_and_value(q_values)
    
    
    return action, action_q_value
    
def get_best_action_and_value(q_values):
    
    '''
    Devuelve la acción con el Q más alto y su valor (Q)
    @ q_values: list --> Lista con los valores de Q predichos por la red neuronal 
    '''
    
    best_action = argmax(q_values) # 0 ó 1
    best_q_value = q_values[best_action] # Q más alto
    
    return best_action, best_q_value


def get_random_action_and_value(q_values):
    
    '''
    Devuelve una acción aleatoria y su valor (Q)
    @ q_values: list --> Lista con los valores de Q predichos por la red neuronal 
    '''
    
    random_action = ENV.action_space.sample()
    q_value_random_action = q_values[random_action]
    
    return random_action, q_value_random_action

In [8]:
from keras.utils import to_categorical
from numpy import expand_dims, argmax
import gym

def reward_from_greedy_episode(network, render=False):
    
    '''
    Ejecutamos la red en el entorno y obtenemos la recompensa final
    acumulada de ese entorno.
    '''
    
    # Iniciamos el entorno
    environment = gym.make('FrozenLake-v0')
    # Obtenemos la primera observación
    obs = environment.reset()
    # Indicamos que el episodio NO ha acabado
    done = False
    # Iniciamos el contador de recompensas a 0
    total_rew = 0
    
    # Mientras el episodio no haya acabado
    while not done:
        
        # Transoformamos una observación a un BATCH de una única observación
        obs_vector = expand_dims(obs, axis=0)
        # Obtenemos el Q de las acciones para el estado actual
        q_actions = network.predict(obs_vector)[0]
        # Tomamos la acción con mayor Q --> política greedy
        action = argmax(q_actions)
        # ejecutamos la acción en el entorno y obtenemos el estado alcanzado, 
        # la recompensa y si el episodio ha terminado
        obs, rew, done, _ = environment.step(action)
        # acumulamos la recompensa obtenida a la recompensa total
        total_rew += rew
        # renderizamos el frame if True
        if render: environment.render() 
    
    # Al terminar cerramos el entorno
    environment.close()
    
    # Devolvemos la recompensa total
    return total_rew

In [9]:
from collections import deque, namedtuple
from random import sample
from numpy import array

class memory_replay:
    
    '''
    Objeto que acumula los siguientes datso en cada paso sobre el entorno:
    - El estado actual en un momento t
    - La acción realizada en ese estado en el momento t
    - El estado alcanzado en el momento t+1
    - La recompensa obtenida por pasar del estado actual al estado alcanzado
    - Un parámetro True/False que determina si el estado alcanzado es un estado terminal (si el episodio ha terminado)
    '''
    
    def __init__(self, length):
        
        '''
        Creamos una lista especial (deque) en la que almacenar los parámetros mencionados antes
        Para mejor organización creamos una tupla con nombres para acceder más fácilmente a cada parámetro
        '''
        
        self.memory_buffer = deque([], maxlen=length)
        self.mem_vector = namedtuple(
            'mem_vector',
            field_names = ['actual_state', 'action', 'next_state', 'reward', 'done']
        )
        
        
    def memorize(self, actual_state, action, next_state, reward, done):
        
        '''
        Almacena cada parámetro en una tupla con nombres y esa tupla en el deque
        '''
        
        vector = self.mem_vector(
            actual_state = actual_state,
            action = action, 
            next_state = next_state, 
            reward = reward, 
            done = done
        )
        
        self.memory_buffer.append(vector)
        
    def reset(self):
        
        '''
        formatea el buffer de memoria
        '''
        
        self.memory_buffer.clear()
      
    def buffer_size(self):
        
        '''
        Devuelve el número de instancias guardadas hasta el momento
        '''
        
        return len(self.memory_buffer)
        
    def get_batch(self, n):
        
        '''
        Devuelve, de entre todas las instancias almacenadas, un conjunto aleatorio de:
        - Array con los estados actuales
        - Array con las acciones realizadas en esos estados
        - Array con los estados alcanzados por realizar esas acciones en esos estados actuales
        - Array con las recompensas obtenidas por alcanzar dichos estados 
        - Array con valores True/False que indican si el episodio se ha terminado en el estado alcanzado
        @ n: int --> Número de instancias que queremos devolver
        '''
        
        # Sanity Check: evitamos pedir más instancias de las que tenemos almacenadas
        
        if n > len(self.memory_buffer):
            
            raise Exception("No hay suficientes instancias: se solicitaron {} y hay {}".format(n , len(self.memory_buffer)) )
        
        # Creamos otra namedtuple temporal para organizar los arrays mencionados dentro de un único objeto
        
        mem_batch = namedtuple(
            'mem_batch',
            field_names = ['actual_states', 'actions', 'next_states', 'rewards', 'dones']
        )
        
        # Tomamos de la memoria un número aleatorio n de instancias
        
        instances = sample(self.memory_buffer, n)
        
        # De esas instancias extraemos: sus estados actuales, acciones, estados alcanzados, recompensas 
        # y si el episodio terminó o no en el estado alcanzado
        
        actual_state_sample = array([instance.actual_state for instance in instances])  # Array con los estados actuales
        action_sample = array([instance.action for instance in instances]) # Array con las acciones realizadas en esos estados
        next_state_sample = array([instance.next_state for instance in instances]) # Array con los estados alcanzados
        reward_sample = array([instance.reward for instance in instances]) # Array con las recompensas obtenidas
        done_sample = array([instance.done for instance in instances]) # Array True/False si el episodio ha terminado
        
        # Empaquetamos los arrays en un único objeto para mejorar la transcripción del código
        
        batch = mem_batch(
            actual_states = actual_state_sample, 
            actions = action_sample, 
            next_states = next_state_sample, 
            rewards = reward_sample, 
            dones = done_sample
        )
        
        # Devolvemos el Batch
        
        return batch
        
        

In [13]:
from numpy import expand_dims, copy, mean
from keras.losses import mean_squared_error
import gym

'''
PASO 0: Creamos el entorno, determinamos el número de épocas y el número de episodios a realizar en cada época.
Creamos Epsilon y su factor de descuento. Determinamos el tamaño del Batch de entrenamiento. Y finalmente
determinamos Gamma.

'''

# Creamos el entorno
ENV = gym.make('FrozenLake-v0')

# Definimos el número de épocas y episodios
EPOCS = 20
EPISODES = 50

# Definimos Epsilon (que guía la política epsilon-greedy)
# Y como se irá reduciendo con el tiempo
EPSILON = 1
EPSILON_DECAY = 0.05
EPSILON_MIN = 0.01

# Definimos el tamaño del Batch que usaremos en la fase de entrenamiento
# y el número de épocas que se entrenará la red sobre ese Batch
TRAINING_BATCH_SIZE = 128
TRAINING_EPOCH = 32
# TRAINING_VALIDATION_SPLIT = 0.1

# Factor Gamma de Descuento (en clases anteriores vimos que es una forma de 
# reducir las expectativas de recompensa que esperamos obtener a largo plazo)
GAMMA = 0.9

print('[*] Iniciando Simulación')
print('[*] Número de épocas:', EPOCS)
print('[*] Número de episodios por época:', EPISODES)

# Creamos las dos redes "gemelas" que definen las estimaciones de Q en s (Q_network_local)
# y las estimaciones de Q en s' (Q_network_target)
q_network_local = q_network()
q_network_target = q_network()

# Inicializamos la memoria 
memory = memory_replay(20000)

# Una lista vacía donde almacenar todo el progreso de la red en el entorno de pruebas
training_rewards = []

'''
PASO 1: INTERACCIÓN
Por cada época se ejecuta, primero, un número determinado de episodios, tras cada episodio se determina si hay
suficientes instancias para entrenar la red "local", si es así iniciamos el entrenamiento de esa red. Al terminar
el episodio actualizamos la red "objetivo".
'''

for epoc in range(EPOCS):
    
    print("We are at epoc:", epoc+1, "Epsilon", EPSILON)
    
    # Reducción de epsilon tras cada época
    if EPSILON > EPSILON_MIN:
        EPSILON -= (EPSILON*EPSILON_DECAY)
    
    # Reiniciamos la memoria a 0
    memory.reset()

    # Una lista vacía donde almacenar el progreso de la red en el entorno de pruebas tras cada época
    epoc_rewards = []

    # Por cada episodio dentro de la época actual
    for episode in range(EPISODES):
        
        # Determinamos que el episodio NO ha acabado
        episode_is_done = False
        
        # Reseteamos el entorno y obtenemos nuestra primera observación (observación == estado)
        actual_state = ENV.reset()
        
        # Mientras el episodio no haya terminado
        while not episode_is_done:
            
            # Ampliamos la dimensión de las observaciones para que puedan ser procesadas por la red.
            # Recordemos que las redes en Keras admiten CONJUNTOS de instancias y NO Instancias sueltas.
            # Por Ejemplo: la observación actual (== estado actual) podría ser en este entorno:
            # (-0.00512, 1.25848, 0.222658, 28.54845) --> variables continuas (nos "da igual" qué representa cada valor)
            # la observación actual tiene un "shape" de (4,) pero keras solo acepta BATCHES (conjuntos de observaciones)
            # por lo que debemos transformar su "shape" a (1, 4) lo cual se traduce como un BATCH de UNA ÚNICA INSTANCIA (4,)
            # es decir, un conjunto de instancias formado por UNA ÚNICA INSTANCIA.
            actual_state = to_categorical(
                actual_state, 
                num_classes=ENV.observation_space.n
            )
            actual_state_batch = expand_dims(actual_state, axis=0)
            
            # Obtenemos el valor de Q de las acciones en el estado acual gracias a neustra q_network_local
            # De nuevo Keras nos devuelve un CONJUNTO de PREDICCIONES, como sólo hemos "pasado" una observación
            # obtenemos tan sólo una predicción, de ahí que nos quedemos con la primera (y única) '[0]'
      
            q_values_actual_state = q_network_local.predict(actual_state_batch)[0]
            
            # Obtenemos la acción a realizar siguiendo una política Epsilon-Greedy
            
            selected_action, q_selected_action = get_action_and_value_epsilon_greedy(q_values_actual_state)
            
            # Una vez seleccionada la acción a realizar la aplicamos sobre el entorno y obtenemos
            # el estado alcanzado, la recompensa de transición, y si el episodio ha terminado
            
            next_state, reward, episode_is_done, _ = ENV.step(selected_action)
            
            # Guardamos todos los valores en nuetra memoria para usarlos en el entrenamiento
            
            memory.memorize(actual_state, selected_action, next_state, reward, episode_is_done)
            
            # El estado alcanzado es ahora el estado actual, y repetimos el loop hasta que termine el episodio.
            
            actual_state = next_state
            
        
        '''
        PASO 1: ENTRENAMIENTO
        Tras terminar la fase de interacción llega la fase de entrenamiento donde entrenamos UNA de las dos redes.
        La red a entrenar es SIEMPRE la local, la red OBJETIVO no se actualiza (aún) ya que de ella obtenemos las 
        expectativas de recompensa a futuro Q(s', a(max)). Si actualizáramos las dos a la vez nuestro "objetivo"
        cambiaría con cada entrenamiento y eso desestabilizaría dicho entrenamiento. Este último punto es algo más
        difícil de entender, pero es la clave del algoritmo DQN, por lo que os aconsejo que intentéis interiorizarlo
        para comprender la clase entera.
        
        '''
        
        # Entrenamos sólo si hay instancias suficientes
        if memory.buffer_size() >= TRAINING_BATCH_SIZE:
            
            # Obtenemos un batch aleatorio de instancias recolectadas durante la fase de interacción
            batch = memory.get_batch(TRAINING_BATCH_SIZE)

            actual_states = batch.actual_states
            actions = batch.actions
            next_states = batch.next_states
            rewards = batch.rewards
            dones = batch.dones
            
            # Obtenemos los valores de Q para el vector de estados actuales. Por cada estado actual obtenemos
            # una lista con dos valores (para el entorno CARTPOLE), esos dos valores determinan el Q de cada una
            # de las 2 acciones posibles dentro de cada uno de esos estados.
            q_estimation = q_network_local.predict(actual_states) 
            
            # Obtenemos los valores de Q para el vector de estados alcanzados. Por cada estado alcanzado obtenemos
            # una lista con dos valores (para el entorno CARTPOLE), esos dos valores determinan el Q de cada una
            # de las 2 acciones posibles dentro de cada uno de esos estados.
            q_target = q_network_target.predict(next_states)
            
            # De esos 2 Q nos quedamos con el más alto siguiendo las directrices del algoritmo Q-learning
            q_target = q_target.max(1)
            
            # Si el estado alcanzado del que obtenemos Q es un estado FINAL entonces su Q = 0 !!! 
            # Recordamos: Q es el valor de una acción, determina la cantidad de recompensa que esperamos 
            # obtener a largo plazo desde un estado cualquiera realizando dicha acción. Al ser un estado TERMINAL
            # la expectativa de recompensas DEBE SER 0, ya que el episodio HA TERMINADO, y no podemos obtener
            # nuevas recompensas.
            q_target[dones] = 0 # donde done == True, q_target = 0
            
            # Finalmente aplicamos BELLMAN para determinar el objetivo.
            # Recordamos: lo llamamos objetivo porque idealmente queremos que nuestra red LOCAL (ESTIMACIÓN)
            # nos de como resultado dicho OBJETIVO. Por esto precisamente NO actualizamos la red OBJETIVO tras 
            # cada episodio, porque necesitamos que el OBJETIVO sea EL MISMO durante la fase de entrenamiento
            # de la red LOCAL, para no "liar" a la red local.
            
            target = rewards + GAMMA * q_target # lista de valoroes objetivo para cada instancia
            
            # El siguiente paso idealmente sería algo así:
            # loss = MEANSQUAREDERROR(target - q_estimation(actions))
            # q_network_local.backpropagation(loss)
            # Pero Keras NO permite esto de una forma relativamente sencilla, por lo que, para no hacer más complejo
            # el código usaremos un "SMART-TRICK" jugando con la función de pérdida "loss":
            
            # Vamos a tomar el vector de estimaciones de Q para los estados actuales, los cual nos dan el valor de Q
            # para cada una de las dos acciones posibles para cada uno de esos estados, y vamos a cambiar el valor de Q 
            # de la acción que realizamos en su momento por el valor del objetivo.
            
            Y = copy(q_estimation) # copia de los resultados de q_network_local para los estados actuales del batch
            
            for index, action in enumerate(actions): # Por cada acción dentro del conjunto de acciones del batch
                
                Y[index, action] = target[index] # Cambiamos el valor Y para la acción realizada para que sea = target
            
            # Al introducir de nuevo durante el 
            # entrenamiento el estado actual como INPUT obtendremos los valores de Q para el estado actual, dichos valores
            # son idénticos a los que teníamos en q_estimation, idénticos todos, salvo el de la acción ejecutada que ahora
            # es igual al target del estado alcanzado. Por tanto, cuando se aplique la función de pérdida "MSE"
            # que recordemos que es (target - q_estimation(actions))^2 --> (target - q_estimation) será 0 para todas las 
            # acciones menos para la acción realizada por lo que los pesos se actualizarán UNICAMENTE para la acción realizada.
            
            q_network_local.fit(actual_states,
                         Y,
                         batch_size=TRAINING_BATCH_SIZE,
                         epochs=TRAINING_EPOCH,
                         verbose=0)
            
            # Al final, tras el entrenamiento, copiamos los pesos de la q_network_local a la q_network_target
        
            q_network_target.set_weights(q_network_local.get_weights())
    
        '''
        OPCIONAL: controlar el rendimiento de nuestra red (nuestro agente) en un entorno de pruebas
        
        '''
        
        # Probamos nuestra red en el entorno durante un episodio
        episode_reward = reward_from_greedy_episode(q_network_local, render=False) # min: 0, max: 200
        
        # Incorporamos la recompensa de prueba del episodio al historial de la época
        epoc_rewards.append(episode_reward)
        
    # Incorporamos las recompensas de prueba de cada epoca al historial total
    training_rewards.extend(epoc_rewards)
    
    print("The best reward at EPOC {} is {} with mean {}".format(epoc+1, max(epoc_rewards), mean(epoc_rewards)))



[*] Iniciando Simulación
[*] Número de épocas: 20
[*] Número de episodios por época: 50
Model: "Fully-Connected-Network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Hidden (Dense)               (None, 128)               2176      
_________________________________________________________________
Output (Dense)               (None, 4)                 516       
Total params: 2,692
Trainable params: 2,692
Non-trainable params: 0
_________________________________________________________________
Model: "Fully-Connected-Network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Hidden (Dense)               (None, 128)               2176      
_________________________________________________________________
Output (Dense)               (None, 4)                 516       
Total params: 2,692
Trainable params: 2,692
Non-trai

ValueError: Error when checking input: expected Hidden_input to have shape (16,) but got array with shape (1,)

In [14]:
import matplotlib.pyplot as plt
plt.plot(training_rewards)